In [1]:
from fun.selection_of_ROIs import *

import warnings
warnings.filterwarnings("ignore")

## 1. Specify the folders that will be processed and clean them (remove precedent processing results)

In [2]:
# 1. select all the fresh measurements
data_folder_path = './data/fixation_over_time'
path_folder_temp = './temp'
path_align_folder = './alignment/to_align'
base_dirs = []

for folder in os.listdir(data_folder_path):
    if 'T0_' in folder:
        base_dirs.append(os.path.join(data_folder_path, folder))
        
    # 2. remove previously acquired data to allow the selection of new ROIs
    path_results = os.path.join(data_folder_path, folder, 'polarimetry/550nm/50x50_images')
    for filename in os.listdir(path_results):
        file_path = os.path.join(path_results, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
        except Exception as e:
            print('Failed to delete %s. Reason: %s' % (file_path, e))
                
# 3. clean the alignment folder
for fname in os.listdir(path_align_folder):
    file_path = os.path.join(path_align_folder, fname)
    try:
        if os.path.isfile(file_path) or os.path.islink(file_path):
            os.unlink(file_path)
        elif os.path.isdir(file_path):
            shutil.rmtree(file_path)
    except Exception as e:
        print('Failed to delete %s. Reason: %s' % (file_path, e))

## 2. Loop over all the folders that will be processed

#### 2.1. set global variables (such as the number of squares)

In [3]:
# to perform propagation
number_of_random_squares = 25
fixation = True
wavelength = '550'
square_size = 20
square_number = 1
type_rec_sq = 'square'

# for the propagation data collection
new_folder_names = ['-T0-D12_', '-T0-D24_', '-T0-D36_', '-T0-D48_', '-T0-D7d_']
new_dates = ['2022-05-03', '2022-05-03', '2022-05-04', '2022-05-04', '2022-05-11']
old_folder_name = '-T0_'
old_date = '2022-05-02'

#### 2.2. loop over all the folders that needs to be processed

In [ ]:
# tissue_types = ['WM', 'GM']
tissue_types = ['WM', 'GM']
for path_folder in tqdm(base_dirs):
    for tissue_type in tissue_types:
        
        # 1. set up the parameters and load the Mueller Matrix
        WM = tissue_type == 'WM'
        mask_matter, grid = get_mask_matter_and_grid(path_folder, tissue_type)
        MM_maps = load_data_mm(path_folder, wavelength)[:-1]
        mat = load_data_mm(path_folder, wavelength)[-1]
        path_folder_50x50 = path_folder + '/polarimetry/' + str(wavelength) + 'nm/50x50_images/'
        folder_name = path_folder.split('\\')[-1]
        if WM:
            new_folder_name = 'WM_1'
        else:
            new_folder_name = 'GM_1'
        propagated = False
        
        # 2. automatic selection and preparation for the propagation of the ROIs
        propagation_list = square_selection(path_folder_temp, path_folder, path_folder_50x50, folder_name, wavelength, 
                     number_of_random_squares, square_size, type_rec_sq, WM, mask_matter, grid, 
                     MM_maps, mat)
        
        
        # 3. actually do the alignment
        propagation_list = generate_combined_mask(propagation_list)
        do_alignment()
        output_folders = move_computed_folders()
        
        # 4. and collect the data after propagation
        collect_data_propagated(WM, new_folder_names, new_dates, old_folder_name, old_date, path_folder, 
                        propagation_list, output_folders)

 75%|██████████████████████████████████████████████████████████████▎                    | 3/4 [39:00<13:03, 783.94s/it]

## 3. Show the histograms (Figure 2 for the paper)

In [ ]:
folder = r'./data/fixation_over_time'
new_folder_name = 'WM_1'

In [ ]:
mat

In [ ]:
for f in [os.listdir(folder)[0], os.listdir(folder)[8]]:
    
    if 'T0_' in f:
        
        path_res = os.path.join(folder, f, 'polarimetry', str(wavelength) + 'nm', '50x50_images', new_folder_name)
        mat = load_data_mm(os.path.join(folder, f), wavelength)[-1]
        
        with open(os.path.join(path_res, 'coordinates.txt')) as f:
            coordinates = f.readlines()

        xs, ys = [], []
        for idx, coord in enumerate(coordinates):
            num_coord = int(coord.replace('\n', ''))
            if idx <= 1:
                ys.append(num_coord)
            else:
                xs.append(num_coord)
        data_all = parameters_histograms(mat, xs, ys, path_res)
    
    else:
        
        path_res = os.path.join(folder, f, 'polarimetry', str(wavelength) + 'nm', '50x50_images', new_folder_name + '_align')
        with open(os.path.join(path_res, 'data_raw.pickle'), 'rb') as handle:
            raw_data = pickle.load(handle)

In [ ]:
linear_retardance_before = data_all['Linear retardance (°)']
linear_retardance_after = raw_data[0][0][3]

depolarization_before = data_all['Depolarization']
depolarization_after = raw_data[0][3][3]

azimuth_before = data_all['Azimuth of optical axis (°)']
azimuth_after = raw_data[0][2][3]

In [ ]:
keys = ['Linear retardance (°)', 'Depolarization', 'Azimuth of optical axis (°)']
parameters = [[linear_retardance_before, linear_retardance_after], [depolarization_before, depolarization_after],
             [azimuth_before, azimuth_after]]

In [ ]:
fig, axes = plt.subplots(nrows=3, ncols=1, figsize=(10,14))

for i, key, param in zip(range(0,3), keys, parameters):
    row = i
    ax = axes[row]
    
    # change the range of the histograms
    if key == 'Depolarization':
        range_hist = (0.85, 1)
    elif key == 'Linear retardance (°)':
        range_hist = (0, 60)
    else:
        range_hist = (0, 180)
        
        
    for idx_p, vals in enumerate(param):
        y, x = np.histogram(
            vals,
            bins=50,
            density=False,
            range = range_hist)
        
        x_plot = []
        for idx, x_ in enumerate(x):
            try: 
                x_plot.append((x[idx] + x[idx + 1]) / 2)
            except:
                pass
            
        # get the mean, max and std
        max_ = x[np.argmax(y)]
        if key == 'Azimuth of optical axis (°)':
            mean = average_angles(vals)
            differences = []
            for v in vals:
                differences.append(subtract_angle(mean, v) * subtract_angle(mean, v))
            std = np.sqrt(np.sum(differences)/len(differences))
        else:
            mean = np.nanmedian(vals)
            std = np.nanstd(vals)
        
        if idx_p == 0:
            mean_bf = mean
        else:
            mean_af = mean
            
        y = y / np.max(y)
        
        # plot the histogram
        if idx_p == 0:
            color = '#52a736'
            line1, = ax.plot(x_plot, y, c = color, linewidth=3)
        else:
            color = '#FF3030'
            line2, = ax.plot(x_plot, y, c = color, linewidth=3)

        
        ax.axis(ymin=0,ymax=1.5)
        ax.locator_params(axis='y', nbins=4)
        ax.locator_params(axis='x', nbins=5)
        
        
        if idx_p == 0:
            if key == 'Azimuth of optical axis (°)':
                ax.text(0.70, 0.52, '$\mu_{ROI}$' + ' = {:.2f}\n'.format(mean) + '$\sigma_{ROI}$' + ' = {:.2f}'.format(std), 
                    horizontalalignment='center', verticalalignment='center', transform=ax.transAxes, 
                    fontsize=24, fontweight = 'bold', c = color)
            else:
                if key == 'Depolarization':
                    ax.text(0.85, 0.52, '$m_{ROI}$'+ ' = {:.2f}\n'.format(mean) + '$\sigma_{ROI}$'+' = {:.2f}'.format(std), 
                            horizontalalignment='center', verticalalignment='center', transform=ax.transAxes, 
                            fontsize=24, fontweight = 'bold', c = color)
                else:
                    ax.text(0.85, 0.52, '$m_{ROI}$'+ ' = {:.1f}\n'.format(mean) + '$\sigma_{ROI}$'+' = {:.1f}'.format(std), 
                            horizontalalignment='center', verticalalignment='center', transform=ax.transAxes, 
                            fontsize=24, fontweight = 'bold', c = color)

        else:
            if key == 'Azimuth of optical axis (°)':
                ax.text(0.32, 0.54, '$\mu_{ROI}$' + ' = {:.2f}\n'.format(mean) + '$\sigma_{ROI}$' + ' = {:.2f}'.format(std), 
                    horizontalalignment='center', verticalalignment='center', transform=ax.transAxes, 
                    fontsize=25, fontweight = 'bold', c = color)
            else:
                ax.text(0.85, 0.75, '$fc_{ROI}$'+ ' = {:.2f}\n'.format(mean / mean_bf), 
                        horizontalalignment='center', verticalalignment='center', transform=ax.transAxes, 
                        fontsize=25, fontweight = 'bold')

                if key == 'Depolarization':
                    ax.text(0.17, 0.54, '$m_{ROI}$'+ ' = {:.2f}\n'.format(mean) + '$\sigma_{ROI}$'+' = {:.2f}'.format(std), 
                        horizontalalignment='center', verticalalignment='center', transform=ax.transAxes, 
                        fontsize=25, fontweight = 'bold', c = color)
                
                else:
                    ax.text(0.17, 0.54, '$m_{ROI}$'+ ' = {:.1f}\n'.format(mean) + '$\sigma_{ROI}$'+' = {:.1f}'.format(std), 
                        horizontalalignment='center', verticalalignment='center', transform=ax.transAxes, 
                        fontsize=25, fontweight = 'bold', c = color)
                
            
    for tick in ax.xaxis.get_major_ticks():
        tick.label1.set_fontsize(28)
        tick.label1.set_fontweight('bold')
    for tick in ax.yaxis.get_major_ticks():
        tick.label1.set_fontsize(28)
        tick.label1.set_fontweight('bold')
            
    # ax.set_title(param[0], fontdict = {'fontsize': 30, 'fontweight': 'bold'})
    # ax.set_ylabel('Normalized pixel number', fontdict = {'fontsize': 20, 'fontweight': 'bold'})
        
    # save the figures
    plt.tight_layout()
    legend_properties = {'weight':'bold', 'size': 25}

    leg = ax.legend([line1, line2], ['Fresh tissue', 'Fixed tissue'], prop=legend_properties, loc = 2)
    leg.get_frame().set_linewidth(0.0)
    
plt.savefig(os.path.join(path_res, 'summary_histo.pdf'))